### Single model run
Obtain the highest posterior parameter set from the selected analysis. Then illustrate results with basic plots, including the matrix scaling.

### Running over Colab
Uncomment the commands in the following cell to install in Colab.
Once completed, click the 'Restart runtime' button that will appear to restart the Colab environment and then proceed to the following cells.

### Generating documentation
Running the final cell will take several minutes and will generate the corresponding PDF documentation.
This can be downloaded from the Files section of the sidebar.

In [ ]:
import pandas as pd
import pymc as pm
import arviz as az

from estival.sampling.tools import idata_to_sampleiterator
from summer2.parameters import Parameter, DerivedOutput, Function

from inputs.constants import SUPPLEMENT_PATH, PRIMARY_ANALYSIS, RUNS_PATH, RUN_IDS
from aust_covid.inputs import get_ifrs
from aust_covid.model import build_model
from aust_covid.calibration import get_priors, get_targets
from aust_covid.plotting import plot_single_run_outputs, plot_example_model_matrices, plot_infection_processes
from emutools.tex import DummyTexDoc, StandardTexDoc, add_image_to_doc
from emutools.utils import load_param_info

In [ ]:
# !pip uninstall numba -y
# !pip uninstall librosa -y
# !pip install estival==0.4.9 numpy==1.24.3 kaleido

In [ ]:
try:
    import google.colab
    on_colab = True
    ! git clone https://github.com/monash-emu/aust-covid.git --branch main
    %cd aust-covid
    %pip install -e ./
    import multiprocessing as mp
    mp.set_start_method('forkserver')
except:
    on_colab = False

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'single_run', 'Single run', 'austcovid')
param_info = load_param_info()
param_info['value'].update(get_ifrs(DummyTexDoc()))
parameters = param_info['value'].to_dict()
intro_text = 'Results from an example run are presented in Figure \\ref{single_run_outputs}, and model construction is described in the following sections. '
app_doc.add_line(intro_text, 'Introduction')
epi_model = build_model(app_doc, param_info['abbreviations'], mobility_ext=True, cross_ref=False)
priors = get_priors(False, param_info['abbreviations'], DummyTexDoc())
prior_names = [p.name for p in priors]
targets = get_targets(DummyTexDoc())

In [ ]:
analysis_folder = RUN_IDS[PRIMARY_ANALYSIS]
i_max = pd.read_hdf(RUNS_PATH / analysis_folder / 'output/results.hdf', 'likelihood')['logposterior'].idxmax()
idata = az.from_netcdf(RUNS_PATH / analysis_folder / 'output/calib_full_out.nc')
best_params = idata_to_sampleiterator(idata).convert('pandas').loc[i_max].to_dict()
parameters.update(best_params)

In [ ]:
epi_model.run(parameters=parameters)

In [ ]:
outputs_fig = plot_single_run_outputs(epi_model, targets)
outputs_fig

In [ ]:
example_matrix_fig = plot_example_model_matrices(epi_model, parameters)
example_matrix_fig.update_layout(height=850)

In [ ]:
infection_fig = plot_infection_processes(epi_model.get_derived_outputs_df(), targets, 'notifications_ma')
infection_fig

In [ ]:
add_image_to_doc(outputs_fig.update_layout(showlegend=False), 'single_run_outputs', 'svg', 'Outputs from single model run', app_doc, 'Outputs')
add_image_to_doc(example_matrix_fig.update_layout(height=520), 'example_matrices', 'svg', 'Dynamic mixing matrices', app_doc, 'Mixing')
caption = 'Colour shows infection with BA.1 (greens), BA.2 (blues) and BA.5 (purples). ' \
    'Shading depth shows infection process, with initial infection (dark), early reinfection (intermediate darkness), late reinfection (light).' \
    'Note early reinfection with BA.1 does not occur to a significant extent. '
add_image_to_doc(infection_fig.update_layout(showlegend=False), 'infect_process', 'svg', 'Simulated infection processes for maximum likelihood results', app_doc, 'Outputs', caption=caption)

In [ ]:
app_doc.write_doc()

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install inkscape texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir(SUPPLEMENT_PATH)
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex -shell-escape single_run
    ! biber single_run
    ! pdflatex -shell-escape single_run
    ! pdflatex -shell-escape single_run